ICESat-2 orientation can be: backward, forward, and transition. During transition, ICESat-2 is maneuvering between the two orientations. Science quality is potentially degraded while in transition mode. (Source: https://nsidc.org/sites/nsidc.org/files/technical-references/ICESat2_ATL03_data_dict_v005.pdf)


The parameter "/orbit_info/sc_orient" in ATL03 denotes the orientation of ICESat-2.

0 = backward

1 = forward

2 = transition

User input: "strong" or "weak"

Script reads the variable "/orbit_info/sc_orient" to determine orientation.

In [7]:
import h5py
import pandas as pd

In [ ]:
# If input = "strong":
#     If sc_orient = 0:
#         subset gt1l, gt2l, gt3l
#     elif sc_orient = 1:
#         subset gt1r, gt2r, gt3r
#     else:
#         print("Error: transition phase"), or other output?

In [ ]:
def get_strong_beams:
    if sc_orient = 0:
        # subset gt1l, gt2l, gt3l
    elif: sc_orient = 1:
        # subset gt1r, gt2r, gt3r
    else:
        print("Error: transition phase") # or other output?

In [ ]:
# If input = "weak":
#     If sc_orient = 0:
#         subset gt1r, gt2r, gt3r
#     elif sc_orient = 1:
#         subset gt1l, gt2l, gt3l
#     else:
#         print("Error: transition phase"), or other output?

In [ ]:
def get_weak_beams:
    if sc_orient = 0:
        # subset gt1r, gt2r, gt3r
    elif sc_orient = 1:
        # subset gt1l, gt2l, gt3l
    else:
        print("Error: transition phase") # or other output?


In [20]:
is2_file = 'processed_ATL07-01_20210223050453_09351001_005_01.h5'
with h5py.File(is2_file, 'r') as f:
    is2_gt2r = pd.DataFrame(data={'lat': f['gt2r/sea_ice_segments/latitude'][:],
                                  'lon': f['gt2r/sea_ice_segments/longitude'][:],
                                  'elev': f['gt2r/sea_ice_segments/heights/height_segment_height'][:]
                                 }
                           ) # Central weak beam
    # is2_gt2l = pd.DataFrame(data={'lat': f['gt2l/sea_ice_segments/latitude'][:],
    #                               'lon': f['gt2l/sea_ice_segments/longitude'][:],
    #                               'elev': f['gt2l/sea_ice_segments/heights/height_segment_height'][:]
    #                              }
    #                        ) # Central strong beam
    # orient = pd.DataFrame(data={'lat': f['gt2r/sea_ice_segments/latitude'][:],
    #                             'lon': f['gt2r/sea_ice_segments/longitude'][:],
    #                             'orient': f['orbit_info/sc_orient'][:]
    #                            }
    #                      )

KeyError: 'Unable to open object (component not found)'

,orient
0,1
